<a href="https://colab.research.google.com/github/wenhaoxie/Home-Credit-Default-Risk/blob/master/webCrawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install scrapy

In [0]:
import scrapy
import logging
import pandas as pd
from google.colab import files
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings
from twisted.internet import reactor
import scrapy
import scrapy.crawler as crawler
from multiprocessing import Process, Queue

In [0]:
class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'http://quotes.toscrape.com/tag/humor/',
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'FEED_FORMAT':'csv',                                 # Used for pipeline 2
        'FEED_URI': 'tes.csv'                        # Used for pipeline 2
    }
    def parse(self, response):
        for quote in response.xpath('//div[@class="quote"]'):
            yield {
                'text': quote.xpath('span[@class="text"]/text()').extract_first(),
                'author': quote.xpath('span/small[@class="author"]/text()').extract_first(),
            }

        next_page = response.xpath('//li[@class="next"]/@herf').extract_first()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

In [0]:

def run_spider(spider):
    def f(q):
        try:
            runner = crawler.CrawlerRunner()
            deferred = runner.crawl(spider)
            deferred.addBoth(lambda _: reactor.stop())
            reactor.run()
            q.put(None)
        except Exception as e:
            q.put(e)
    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    result = q.get()
    p.join()

    if result is not None:
        raise result

In [0]:
run_spider(QuotesSpider)

In [0]:
ll 

total 12
drwxr-xr-x 1 root 4096 Apr 29 16:32 sample_data/
-rw-r--r-- 1 root 1184 May  4 00:13 tes.csv
-rw-r--r-- 1 root 1184 May  4 00:13 tesst.csv


In [0]:

df_test = pd.read_csv('tes.csv')
df_test.head()

,text,author
0,"“The person, be it gentleman or lady, who has ...",Jane Austen
1,"“A day without sunshine is like, you know, nig...",Steve Martin
2,“Anyone who thinks sitting in church can make ...,Garrison Keillor
3,“Beauty is in the eye of the beholder and it m...,Jim Henson
4,“All you need is love. But a little chocolate ...,Charles M. Schulz


In [0]:
files.download('tes.csv')

In [0]:
class BagSpider(scrapy.Spider):
    name = "bag_review"
    start_urls = [
        'https://www.amazon.com/Backpack-Business-Charging-Resistant-Computer/product-reviews/B06XZTZ7GB/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'FEED_FORMAT':'csv',                                 # Used for pipeline 2
        'FEED_URI': 'review.csv'                        # Used for pipeline 2
    }
    def parse(self, response):
        for quote in response.xpath('//*[@id="cm_cr-review_list"]/div/div/div/'):
            yield {
                
                
                'text': quote.xpath('div[4]/span/span/text()').extract_first(),
                
            }

        #next_page = response.xpath('//li[@class="next"]/@herf').extract_first()
        #if next_page is not None:
        #    next_page = response.urljoin(next_page)
        #    yield scrapy.Request(next_page, callback=self.parse)

In [0]:
run_spider(BagSpider)

In [6]:
df_test = pd.read_csv('review.csv')
df_test.head()

EmptyDataError: ignored